# Python SQL
## - CRUD Operation

In [1]:
import json
import pymysql

In [4]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [5]:
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port'],
)

## ★ 테이블 생성

In [6]:
sql_user = """
    CREATE TABLE if NOT EXISTS users (
	uid VARCHAR(20) NOT NULL PRIMARY KEY,
	pwd CHAR(44),
	uname VARCHAR(20) NOT NULL,
	reg_date DATETIME DEFAULT CURRENT_TIMESTAMP,
	is_deleted INT DEFAULT 0
);
"""

In [7]:
cur = conn.cursor()
cur.execute(sql_user)

0

## ★ 테이블 변경

In [8]:
cur.execute('alter table users add column email varchar(40);')

0

## ★ 데이터 입력

In [15]:
cur = conn.cursor()
cur.execute("INSERT INTO users(uid, uname) VALUES('admin', '관리자');")

IntegrityError: (1062, "Duplicate entry 'admin' for key 'users.PRIMARY'")

In [16]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 10, 52, 2), 0, None)


In [17]:
# 입력한 데이터를 즉시 flush 하려면 conn.commit() 추가
sql_insert_multi = """
    INSERT INTO users(uid, uname) VALUES('eskim', '김은숙'),('wjlee','이우정');
"""
cur.execute(sql_insert_multi)
conn.commit()

In [20]:
sql_select = """
    select uid, uname, date_format(reg_date, '%Y-%n-%d %h:%i') as reg_date,
    is_deleted, email from users;
"""
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', '관리자', '2021-n-09 10:52', 0, None)
('eskim', '김은숙', '2021-n-09 10:55', 0, None)
('wjlee', '이우정', '2021-n-09 10:55', 0, None)


In [22]:
# 파라메터 이용, placeholder
uid = 'djy'
uname = '대조영'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
cur.execute(sql, (uid, uname))
conn.commit()

In [23]:
uid = 'gdhong'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, '홍길동');"
cur.execute(sql, (uid, ))  # uid만 넣을 경우, 튜플이 아니기 때문에 에러
conn.commit()

In [28]:
users = (('jbpark', '박재범'), ('vincenzo','빈센조'))
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
# 아래 for문은 cur.executemany(sql, users)로 대체 가능
for user in users:
    cur.execute(sql, user)
conn.commit()

## ★ 데이터 삭제

In [36]:
cur = conn.cursor()
sql = "DELETE FROM users WHERE uid !='admin';"
cur.execute(sql)
conn.commit()

In [37]:
import pandas as pd
uids = ['diy', 'eskim','gdhong','jbpark','vincenzo','wjlee']
unames = ['대조영','김은숙','홍길동','박재범','빈센조','이우정']
df = pd.DataFrame({'uid':uids,'uname':unames})
df

,uid,uname
0,diy,대조영
1,eskim,김은숙
2,gdhong,홍길동
3,jbpark,박재범
4,vincenzo,빈센조
5,wjlee,이우정


In [38]:
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
for i in df.index:
    cur.execute(sql, (df.uid[i], df.uname[i]))
conn.commit()

## ★ 데이터 수정

In [39]:
email = 'admin@korea.com'
cur = conn.cursor()
sql = "UPDATE users SET email = %s WHERE uid=%s;"
cur.execute(sql, (email, uid))
conn.commit()